<a href="https://colab.research.google.com/github/boneeyah/SMU/blob/main/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 8.2 MB/s 
     |████████████████████████████████| 1.6 MB 38.3 MB/s 


In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [26]:
from pandas.io.parsers.readers import read_csv
df = read_csv("https://raw.githubusercontent.com/boneeyah/SMU/main/netflix_titles.csv")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [28]:
df = df.drop(index=df[df['country'].isnull()].index)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7976 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7976 non-null   object
 1   type          7976 non-null   object
 2   title         7976 non-null   object
 3   director      5751 non-null   object
 4   cast          7305 non-null   object
 5   country       7976 non-null   object
 6   date_added    7967 non-null   object
 7   release_year  7976 non-null   int64 
 8   rating        7973 non-null   object
 9   duration      7973 non-null   object
 10  listed_in     7976 non-null   object
 11  description   7976 non-null   object
dtypes: int64(1), object(11)
memory usage: 810.1+ KB


In [8]:
colors = {
    'background':'#111111',
    'text':'#7fdbff'
}

In [61]:
df13 = df[['country','type','show_id']]

df13 = df13[['country','show_id']].groupby('country').count()
px.scatter_geo(data_frame=df13,locations=df13.index,size='show_id',locationmode='country names')

In [60]:
df13.index

Index([', France, Algeria', ', South Korea', 'Argentina',
       'Argentina, Brazil, France, Poland, Germany, Denmark',
       'Argentina, Chile', 'Argentina, Chile, Peru', 'Argentina, France',
       'Argentina, France, United States, Germany, Qatar', 'Argentina, Italy',
       'Argentina, Spain',
       ...
       'Uruguay, Argentina, Germany, Spain', 'Uruguay, Argentina, Spain',
       'Uruguay, Germany', 'Uruguay, Guatemala', 'Uruguay, Spain, Mexico',
       'Venezuela', 'Venezuela, Colombia', 'Vietnam', 'West Germany',
       'Zimbabwe'],
      dtype='object', name='country', length=748)

In [17]:
app = JupyterDash(__name__)
app.layout = html.Div(style = {'backgroundColor':colors['background']},children = [
    html.H1('Week12',style = {'color':colors['text']})
    
    ])


if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>